In [2]:
import xarray as xr
import rioxarray as rxr
import rasterio
import glob
import geopandas as gpd
from tqdm import tqdm_notebook
import pandas as pd
from rasterstats import zonal_stats

In [3]:
poly = gpd.read_file('D:\Work\WB\LDT\countries\SRB\shapefiles\gadm41_SRB_0.json')
poly.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [4]:
shapefile_path = 'D:\Work\WB\LDT\countries\SRB\shapefiles\gadm41_SRB_0.json'
nc_file_path = r"D:\Work\WB\LDT\data\raw_data\gfdl-esm4_r1i1p1f1_w5e5_ssp585_tasmax_global_daily_2015_2020.nc"

files = glob.glob("D:/Work/WB\LDT/data/raw_data/*.nc")

# Load the shapefile as a GeoDataFrame
gdf = gpd.read_file(shapefile_path)

for file in tqdm_notebook(files):
    # Open the NetCDF dataset using xarray with rioxarray extension
    ds = xr.open_dataset(file)

    # Assign CRS and use rioxarray for spatial operations
    ds.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
    ds.rio.write_crs("EPSG:4326", inplace=True)

    # Clip the dataset using the shapefile
    clipped_ds = ds.rio.clip(gdf.geometry.values, gdf.crs, drop=True, all_touched=True)

    # Save the clipped dataset as a new NetCDF file
    output_clipped_file = file.replace(".nc", ".clipped.nc")
    clipped_ds.to_netcdf(output_clipped_file)

C:\Users\sonle\AppData\Local\Temp\ipykernel_11064\578766444.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for file in tqdm_notebook(files):


  0%|          | 0/9 [00:00<?, ?it/s]

In [7]:
test = rxr.open_rasterio(r"D:\Work\WB\LDT\data\raw_data\gfdl-esm4_r1i1p1f1_w5e5_ssp585_tasmax_global_daily_2015_2020.clipped.nc")
test.sel(time='2020-01-01').rio.to_raster(r"D:\Work\WB\LDT\data\raw_data\test.tiff")

In [5]:
# https://www.isimip.org/gettingstarted/input-data-bias-adjustment/details/84/

files = glob.glob("D:/Work/WB\LDT/data/raw_data/*clipped.nc")

combined_dataset = xr.open_mfdataset(files, combine='by_coords', parallel=False)
combined_dataset.to_netcdf("D:/Work/WB/LDT/data/raw_data/gfdl-esm4_ssp585_tasmax_global_daily_2015_2100_SRB_touched.nc")

In [45]:
import rasterio
import geopandas as gpd
from shapely.geometry import box

# Load the raster file
raster_path = r"D:\Work\WB\LDT\data\raw_data\test.tiff"
with rasterio.open(raster_path) as dataset:
    # Get the bounds of each pixel
    bounds = []
    for row in range(dataset.height):
        for col in range(dataset.width):
            # Get the pixel bounds
            pixel_bounds = dataset.window_bounds(((row, row + 1), (col, col + 1)))
            bounds.append(box(*pixel_bounds))
    
    # Create a GeoDataFrame from the bounds
    gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(bounds), crs=dataset.crs)
gdf = gdf.sjoin(poly).drop(['index_right', 'GID_0', 'COUNTRY'], axis=1).reset_index(drop=True)
gdf['GRID_N'] = gdf.index
# Save the GeoDataFrame as a GeoJSON file
output_json_path = r"D:\Work\WB\LDT\data\raw_data\pixels.json"
gdf[['GRID_N', 'geometry']].to_file(output_json_path, driver='GeoJSON', index=False)

In [46]:
gdf

,geometry,GRID_N
0,"POLYGON ((19.5 46, 19.5 46.5, 19 46.5, 19 46, ...",0
1,"POLYGON ((20 46, 20 46.5, 19.5 46.5, 19.5 46, ...",1
2,"POLYGON ((20.5 46, 20.5 46.5, 20 46.5, 20 46, ...",2
3,"POLYGON ((19 45.5, 19 46, 18.5 46, 18.5 45.5, ...",3
4,"POLYGON ((19.5 45.5, 19.5 46, 19 46, 19 45.5, ...",4
5,"POLYGON ((20 45.5, 20 46, 19.5 46, 19.5 45.5, ...",5
6,"POLYGON ((20.5 45.5, 20.5 46, 20 46, 20 45.5, ...",6
7,"POLYGON ((21 45.5, 21 46, 20.5 46, 20.5 45.5, ...",7
8,"POLYGON ((19 45, 19 45.5, 18.5 45.5, 18.5 45, ...",8
9,"POLYGON ((19.5 45, 19.5 45.5, 19 45.5, 19 45, ...",9


In [5]:
import sys
sys.path.append('D:\Work\WB\LDT')
from src.dhelper import *
from rasterstats import zonal_stats
import concurrent 

dates = get_daterange('2023-01-01', '2100-12-31', 
              input_format='%Y-%m-%d',
              output_format='%Y-%m-%d')

In [6]:
ds = xr.open_dataset("D:/Work/WB/LDT/data/raw_data/gfdl-esm4_ssp585_tasmax_global_daily_2015_2100_SRB_touched.nc")
pixels = gpd.read_file(r"D:\Work\WB\LDT\data\raw_data\pixels.json")

tasmax = ds['tasmax']
affine = tasmax.rio.transform()
grids = pixels['GRID_N'].to_list()

dfs = []

for date in tqdm_notebook(dates[:]):
    tm_d = tasmax.sel(time=date)
    stats = zonal_stats(pixels.geometry, tm_d.values, affine=affine, stats=['mean'])
    df = pd.DataFrame(stats)
    df['date'] = date
    df['GRID_N'] = grids
    dfs.append(df)

C:\Users\sonle\AppData\Local\Temp\ipykernel_34484\1157600090.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for date in tqdm_notebook(dates[:]):


  0%|          | 0/28489 [00:00<?, ?it/s]

c:\Users\sonle\anaconda3\envs\wb-ldt\Lib\site-packages\rasterstats\io.py:328: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


In [1]:
# def calc_zonal_heat(ds, poly, affine, date):
#     tm_d = ds.sel(time=date)
#     stats = zonal_stats(poly.geometry, tm_d.values, affine=affine, stats=['mean'])
#     df = pd.DataFrame(stats)
#     df['date'] = date
#     df['GRID_N'] = poly['GRID_N'].to_list()
#     return df

# with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
#     futures = [executor.submit(calc_zonal_heat, tasmax, pixels, affine, date) for date in dates]

In [11]:
df_tasmax = pd.concat(dfs, axis=0).reset_index(drop=True)
df_tasmax['date'] = pd.to_datetime(df_tasmax['date'])

In [12]:
def count_consecutive_exceedances_all_grids(df, temp_threshold, n_days):
    """
    Counts the number of times the temperature exceeds the specified threshold for n consecutive days 
    for each grid in the DataFrame.

    Parameters:
        df (pd.DataFrame): DataFrame containing "mean", "date", and "grid_n" columns.
        temp_threshold (float): The temperature threshold to compare against.
        n_days (int): The number of consecutive days the temperature should exceed the threshold.

    Returns:
        pd.DataFrame: DataFrame with grid numbers and their corresponding exceedance counts.
    """
    # Initialize an empty list to store results
    results = []

    # Loop through each unique grid number
    for grid in df['GRID_N'].unique():
        # Filter the data for the current grid
        df_grid = df[df['GRID_N'] == grid]
        
        # Sort the DataFrame by date to ensure we are checking consecutive days in order
        df_grid = df_grid.sort_values(by='date')
        
        # Create a boolean mask where temperature exceeds the threshold
        exceed_mask = df_grid['mean'] > temp_threshold + 273.15
        
        # Initialize the count of exceedances
        exceedance_count = 0
        
        # Initialize a counter to track consecutive exceedances
        consecutive_days = 0
        
        # Loop through the boolean mask
        for exceeded in exceed_mask:
            if exceeded:
                # If the temperature exceeded the threshold, increase consecutive count
                consecutive_days += 1
            else:
                # If it didn't exceed, reset the consecutive count
                consecutive_days = 0
            
            # Check if we've reached the required number of consecutive days
            if consecutive_days >= n_days:
                exceedance_count += 1
                # Reset the consecutive days count to avoid double-counting overlapping periods
                consecutive_days = 0
        
        # Append the result for the current grid
        results.append({'GRID_N': grid, 'exceedance_count': exceedance_count})
    
    # Convert results to a DataFrame
    return pd.DataFrame(results)

In [13]:
result_df = count_consecutive_exceedances_all_grids(df_tasmax, temp_threshold=40, n_days=5)
result_df.head()

,GRID_N,exceedance_count
0,0,19
1,1,17
2,2,23
3,3,21
4,4,24


In [44]:
rails = gpd.read_file("D:\Work\WB\LDT\countries\SRB\\raw_data\serbia-230101-free.shp\gis_osm_railways_free_1.shp")
muni = gpd.read_file('D:\Work\WB\LDT\countries\SRB\shapefiles\gadm41_SRB_2.json')
rails = rails.sjoin(pixels).drop(['index_right'], axis=1)
rails = rails.merge(result_df, on=['GRID_N'], how='left')
rails = rails.sjoin(muni[['GID_2', 'geometry']]).drop(['index_right'], axis=1)
rails = rails[rails['exceedance_count'] > 0]
rails_km = rails.to_crs(epsg=3857)
rails_km['rail_length_km'] = rails_km.length / 1000
rails_sum = rails_km.groupby(['GID_2']).agg({'rail_length_km': 'sum'}).reset_index()
rails_sum = pd.merge(muni[['GID_2']], rails_sum, on=['GID_2'], how='left')
rails_sum['rail_length_km'] = rails_sum['rail_length_km'].fillna(0)
rails_sum['year'] = 2022 
rails_sum

,GID_2,rail_length_km,year
0,SRB.1.1_1,91.160238,2022
1,SRB.1.2_1,0.000000,2022
2,SRB.1.3_1,48.239500,2022
3,SRB.1.4_1,85.573661,2022
4,SRB.2.1_1,19.811233,2022
...,...,...,...
156,SRB.25.6_1,60.207639,2022
157,SRB.25.7_1,5.383156,2022
158,SRB.25.8_1,63.383139,2022
159,SRB.25.9_1,10.862056,2022


In [45]:

# rails_sum.to_csv("D:\Work\WB\LDT\countries\SRB\datasets\SRB_heatwave_risk_rails_2022.csv", index=False)

In [46]:
roads = gpd.read_file(r"D:\Work\WB\LDT\countries\SRB\raw_data\serbia-230101-free.shp\gis_osm_roads_free_1.shp")
roads = roads[~roads['fclass'].isin(['cycleway', 'unknown', 'footway', 'pedestrian', 'steps', 'path', 'track_grade2', 'track_grade3', 'track_grade4', 'track_grade5', 'bridleway'])]
muni = gpd.read_file('D:\Work\WB\LDT\countries\SRB\shapefiles\gadm41_SRB_2.json')

roads = roads.sjoin(pixels).drop(['index_right'], axis=1)
roads = roads.merge(result_df, on=['GRID_N'], how='left')
roads = roads.sjoin(muni[['GID_2', 'geometry']]).drop(['index_right'], axis=1)
roads = roads[roads['exceedance_count'] > 0]
roads_km = roads.to_crs(epsg=3857)
roads_km['road_length_km'] = roads_km.length / 1000
roads_sum = roads_km.groupby(['GID_2']).agg({'road_length_km': 'sum'}).reset_index()
roads_sum = pd.merge(muni[['GID_2']], roads_sum, on=['GID_2'], how='left')
roads_sum['road_length_km'] = roads_sum['road_length_km'].fillna(0)
roads_sum['year'] = 2022 
roads_sum

,GID_2,road_length_km,year
0,SRB.1.1_1,2990.901095,2022
1,SRB.1.2_1,1389.346154,2022
2,SRB.1.3_1,2155.660005,2022
3,SRB.1.4_1,2562.409002,2022
4,SRB.2.1_1,825.662548,2022
...,...,...,...
156,SRB.25.6_1,1495.303792,2022
157,SRB.25.7_1,616.241730,2022
158,SRB.25.8_1,2052.946448,2022
159,SRB.25.9_1,2428.834177,2022


In [47]:
roads_sum['road_length_km'].sum()

np.float64(281240.5113763298)

In [63]:
heatwave_df = pd.merge(rails_sum, roads_sum, on=['GID_2', 'year'], how='left')
heatwave_df

,GID_2,rail_length_km,year,road_length_km
0,SRB.1.1_1,91.160238,2022,2990.901095
1,SRB.1.2_1,0.000000,2022,1389.346154
2,SRB.1.3_1,48.239500,2022,2155.660005
3,SRB.1.4_1,85.573661,2022,2562.409002
4,SRB.2.1_1,19.811233,2022,825.662548
...,...,...,...,...
156,SRB.25.6_1,60.207639,2022,1495.303792
157,SRB.25.7_1,5.383156,2022,616.241730
158,SRB.25.8_1,63.383139,2022,2052.946448
159,SRB.25.9_1,10.862056,2022,2428.834177


In [26]:
import ee
import geemap
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [32]:
poly = gpd.read_file('D:\Work\WB\LDT\countries\SRB\shapefiles\gadm41_SRB_2.json')
poly = poly.to_crs(epsg=4326)
gid_2 = poly['GID_2'].to_list()
pop_stats = []

# Define the dataset collection
# https://developers.google.com/earth-engine/datasets/catalog/WRI_Aqueduct_Flood_Hazard_Maps_V2#image-properties
# https://files.wri.org/d8/s3fs-public/aqueduct-floods-methodology.pdf?_gl=1*1u9ay0w*_gcl_au*MTg3NzAwNTI0OC4xNzI0MjQwNTIw
# https://www.usgs.gov/centers/new-jersey-water-science-center/floods-recurrence-intervals-and-100-year-floods 
pops = ee.ImageCollection("projects/sat-io/open-datasets/ORNL/LANDSCAN_GLOBAL").filterDate('2022-01-01', '2022-12-31')\
                                                         .sum()\
                                                         .select('b1')

for i in tqdm_notebook(range(len(poly))):

    r = poly.iloc[i:i+1]
    region = geemap.gdf_to_ee(r).geometry()

    stats = pops.reduceRegion(
                      geometry=region,
                      reducer=ee.Reducer.sum(),
                      scale=1000,  
                      crs='EPSG:4326', 
                      tileScale=14,
                      bestEffort=True
                )
    
    
    try:
        pop = stats.getInfo()['b1']
        pop_stats.append(pop)
        
    except Exception as e:
         print(e)
         continue
    
pop_df = pd.DataFrame({'GID_2': gid_2,
                       'population_sum': pop_stats})
pop_df['year'] = 2022 
    

C:\Users\sonle\AppData\Local\Temp\ipykernel_34484\3522453047.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(poly))):


  0%|          | 0/161 [00:00<?, ?it/s]

In [35]:
pop_df


,GID_2,population_sum,year
0,SRB.1.1_1,36938.533333,2022
1,SRB.1.2_1,16626.023529,2022
2,SRB.1.3_1,13804.294118,2022
3,SRB.1.4_1,28860.266667,2022
4,SRB.2.1_1,6490.898039,2022
...,...,...,...
156,SRB.25.6_1,27803.866667,2022
157,SRB.25.7_1,17834.627451,2022
158,SRB.25.8_1,25052.764706,2022
159,SRB.25.9_1,24215.105882,2022


In [64]:
heatwave_df = pd.merge(heatwave_df, pop_df, on=['GID_2', 'year'], how='left')
heatwave_df['rail_length_heatwave_pcap'] = 1000*heatwave_df['rail_length_km'] / heatwave_df['population_sum']
heatwave_df['road_length_heatwave_pcap'] = 1000*heatwave_df['road_length_km'] / heatwave_df['population_sum']
heatwave_df

,GID_2,rail_length_km,year,road_length_km,population_sum,rail_length_heatwave_pcap,road_length_heatwave_pcap
0,SRB.1.1_1,91.160238,2022,2990.901095,36938.533333,2.467890,80.969677
1,SRB.1.2_1,0.000000,2022,1389.346154,16626.023529,0.000000,83.564549
2,SRB.1.3_1,48.239500,2022,2155.660005,13804.294118,3.494529,156.158655
3,SRB.1.4_1,85.573661,2022,2562.409002,28860.266667,2.965103,88.786740
4,SRB.2.1_1,19.811233,2022,825.662548,6490.898039,3.052156,127.203130
...,...,...,...,...,...,...,...
156,SRB.25.6_1,60.207639,2022,1495.303792,27803.866667,2.165441,53.780426
157,SRB.25.7_1,5.383156,2022,616.241730,17834.627451,0.301837,34.553104
158,SRB.25.8_1,63.383139,2022,2052.946448,25052.764706,2.529986,81.944906
159,SRB.25.9_1,10.862056,2022,2428.834177,24215.105882,0.448565,100.302439


In [65]:
heatwave_df[['GID_2', 'year', 'rail_length_heatwave_pcap', 'road_length_heatwave_pcap']].to_csv("D:\Work\WB\LDT\countries\SRB\datasets\SRB_heatwave_risk_2022.csv", index=False)

In [66]:
heatwave_df.describe()

,rail_length_km,year,road_length_km,population_sum,rail_length_heatwave_pcap,road_length_heatwave_pcap
count,161.000000,161.0,161.000000,161.000000,161.000000,161.000000
mean,63.074638,2022.0,1746.835474,36592.626647,2.074441,84.254317
std,80.470502,0.0,1058.357529,52658.708031,2.608554,68.106141
min,0.000000,2022.0,0.000000,937.603922,0.000000,0.000000
25%,0.000000,2022.0,1024.966369,11576.000000,0.000000,42.591364
50%,42.974898,2022.0,1475.852388,21012.349020,1.222259,76.646853
75%,89.739146,2022.0,2375.754440,39362.850980,3.075098,107.552932
max,467.915560,2022.0,5363.075450,456825.266667,16.753582,648.933669
